In [1]:
!pip install yfinance

In [2]:
!pip install statsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 4.0 MB/s eta 0:00:00


In [3]:
!pip install prophet

#1. Importação das Bibliotecas

Descrição:

importamos todas as bibliotecas necessárias para a nossa análise. Essas bibliotecas nos permitirão realizar diversas tarefas, desde a coleta de dados até a análise e visualização dos resultados das previsões. Aqui estão as importações e suas respectivas finalidades:

In [4]:
import pandas as pd  # Utilizada para manipulação e análise de dados, especialmente em estruturas de DataFrame.
import numpy as np  # Usada para operações numéricas eficientes em arrays e matrizes.
from statsmodels.tsa.seasonal import seasonal_decompose  # Ferramenta para a decomposição sazonal de séries temporais.
import yfinance as yf  # Utilizada para baixar dados financeiros históricos diretamente do Yahoo Finance.
from statsforecast import StatsForecast  # Biblioteca para previsão de séries temporais que inclui diversos modelos.
from statsforecast.models import SeasonalNaive, SeasonalWindowAverage, AutoARIMA  # Modelos específicos para previsão de séries temporais.
import plotly.graph_objects as go  # Ferramenta para criar gráficos interativos com Plotly.
from plotly.subplots import make_subplots  # Ferramenta para criar subplots com Plotly.
from prophet import Prophet  # Biblioteca de previsão de séries temporais desenvolvida pelo Facebook.
import statsmodels.api as sm  # Ferramentas para a estimação e inferência de modelos estatísticos.
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error  # Funções para avaliar a precisão de modelos preditivos.


#2. Carregamento e Visualização Inicial dos Dados

Descrição:

Carregamos os dados históricos do Ibovespa de um arquivo CSV. Em seguida, verificamos a estrutura do DataFrame e visualizamos as primeiras e últimas linhas dos dados para garantir que a importação foi bem-sucedida.

In [5]:
# Carregar os dados históricos do Ibovespa
dados_ibv = pd.read_csv('/content/Dados_Historicos_Ibovespa.csv', parse_dates=['Data'])

<ipython-input-5-dd03b051a314>:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dados_ibv = pd.read_csv('/content/Dados_Historicos_Ibovespa.csv', parse_dates=['Data'])


In [6]:
# Verificar a estrutura do DataFrame
dados_ibv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      1246 non-null   datetime64[ns]
 1   Último    1246 non-null   float64       
 2   Abertura  1246 non-null   float64       
 3   Máxima    1246 non-null   float64       
 4   Mínima    1246 non-null   float64       
 5   Vol.      1246 non-null   object        
 6   Var%      1246 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 68.3+ KB


In [7]:
# Exibir as primeiras 5 linhas do DataFrame
print(dados_ibv.head(5))

        Data   Último  Abertura   Máxima   Mínima   Vol.    Var%
0 2024-07-19  127.616   127.652  128.360  127.413  7,62B  -0,03%
1 2024-07-18  127.652   129.450  129.454  127.523  7,59B  -1,39%
2 2024-07-17  129.450   129.112  129.658  128.741  6,67B   0,26%
3 2024-07-16  129.110   129.322  129.521  128.761  6,82B  -0,16%
4 2024-07-15  129.321   128.898  129.485  128.723  5,83B   0,33%


In [8]:
# Exibir as últimas 5 linhas do DataFrame
print(dados_ibv.tail(5))

           Data   Último  Abertura   Máxima   Mínima   Vol.    Var%
1241 2019-07-25  102.655   104.119  104.440  102.390  5,98M  -1,41%
1242 2019-07-24  104.120   103.707  104.570  103.707  5,02M   0,40%
1243 2019-07-23  103.704   103.949  104.430  103.518  4,18M  -0,24%
1244 2019-07-22  103.949   103.452  104.278  103.452  3,57M   0,48%
1245 2019-07-19  103.452   104.716  104.723  103.388  3,59M  -1,21%


#3. Preparação dos Dados

Descrição:

Preparamos o DataFrame para a análise ao adicionar uma coluna de identificação e ajustar os nomes das colunas para se adequarem ao formato exigido pelos modelos de previsão. Criamos um intervalo completo de datas considerando apenas os dias úteis e preenchemos os valores faltantes com o último valor conhecido.

In [9]:
# Preparar DataFrame
dados_ibv['Chave'] = 'ibv'
df_1 = dados_ibv[['Data', 'Chave', 'Último']]
df_1 = df_1.rename(columns={'Data': 'ds', 'Último': 'y', 'Chave': 'unique_id'})

In [10]:
# Criar intervalo completo de datas
start_date = df_1['ds'].min()
end_date = df_1['ds'].max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='B')  # Frequência de dias úteis

In [11]:
# Reindexar o DataFrame original para incluir todos os dias úteis
df_1 = df_1.set_index('ds').reindex(all_dates).rename_axis('ds').reset_index()

In [12]:
# Preencher valores faltantes com o último valor conhecido (forward fill)
df_1['y'] = df_1['y'].ffill()

In [13]:
# Definir a coluna 'unique_id'
df_1['unique_id'] = 'ibv'

# 4.Visualização da Evolução do Fechamento da Bolsa
Descrição:


Criamos um gráfico interativo utilizando plotly para visualizar a evolução do preço de fechamento do Ibovespa nos últimos cinco anos. Adicionamos a série de dados reais ao gráfico e configuramos o layout para melhor apresentação.

In [14]:
fig = go.Figure()

# Adicionando a série de dados reais
fig.add_trace(go.Scatter(
    x=df_1['ds'],
    y=df_1['y'],
    mode='lines',
    name='Preço',
    line=dict(width=2),
    hovertext=df_1['ds'].dt.strftime('%Y-%m-%d'),  # Mantendo a data completa no hover
    hoverinfo='text+y'  # Mostrando a data completa e o valor no hover
))

# Configurando o layout
fig.update_layout(
    title={
        'text': 'Evolução do Fechamento da Bolsa nos últimos 5 anos',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%Y',  # Exibir apenas o ano no eixo x
        tickangle=-0,
        type='date'
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'
    ),
    legend_title='Legenda',
    width=1280,
    height=720,
)

fig.show()

#5. Decomposição Sazonal da Série Temporal

Descrição:

Filtramos os dados para do ano de 2024 e realizamos a decomposição sazonal da série temporal. Visualizamos os componentes da decomposição: a série real, a tendência, a sazonalidade e os resíduos, utilizando gráficos interativos com plotly.

In [15]:
# Filtrar os dados para o ano de 2024
df_2024 = df_1.loc[df_1['ds'] > '2024-01-01']

# Decompor a série temporal
seasonplot = seasonal_decompose(df_2024.set_index('ds')['y'], model='multiplicative', period=2)

# Criar a figura com subplots
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    subplot_titles=['Série Real', 'Tendência', 'Sazonalidade', 'Resíduos'],
    vertical_spacing=0.1  # Aumentar o espaçamento vertical entre os subplots
)

# Adicionar a série real
fig.add_trace(go.Scatter(
    x=seasonplot.observed.index, y=seasonplot.observed, mode='lines', name='Série Real',
    hovertext=seasonplot.observed.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=1, col=1)

# Adicionar a tendência
fig.add_trace(go.Scatter(
    x=seasonplot.trend.index, y=seasonplot.trend, mode='lines', name='Tendência',
    hovertext=seasonplot.trend.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=2, col=1)

# Adicionar a sazonalidade
fig.add_trace(go.Scatter(
    x=seasonplot.seasonal.index, y=seasonplot.seasonal, mode='lines', name='Sazonalidade',
    hovertext=seasonplot.seasonal.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=3, col=1)

# Adicionar os resíduos
fig.add_trace(go.Scatter(
    x=seasonplot.resid.index, y=seasonplot.resid, mode='lines', name='Resíduos',
    hovertext=seasonplot.resid.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=4, col=1)

# Atualizar o layout do gráfico
fig.update_layout(
    title={
        'text': 'Decomposição Sazonal da Série Temporal',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=900,  # Aumentar a altura total do gráfico
    width=1000,
    showlegend=False,
    margin=dict(t=150)  # Aumentar o espaço no topo para o título
)

# Atualizar títulos dos eixos
fig.update_xaxes(title_text='Data', row=4, col=1)
fig.update_yaxes(title_text='Série Real', row=1, col=1)
fig.update_yaxes(title_text='Tendência', row=2, col=1)
fig.update_yaxes(title_text='Sazonalidade', row=3, col=1)
fig.update_yaxes(title_text='Resíduos', row=4, col=1)

# Ajustar os ticks do eixo x para mostrar todos os meses
fig.update_xaxes(
    dtick="M1",  # Define a frequência dos ticks para 1 mês
    tickformat="%b",  # Formata os ticks para mostrar apenas o mês
    tickangle=0,  # Rotaciona os rótulos do eixo x para melhor legibilidade
    tickmode="linear",
    rangebreaks=[  # Remove os intervalos nos finais de semana
        dict(bounds=["sat", "mon"])
    ]
)

# Mostrar o gráfico
fig.show()

##6. Definição da Função WMAPE



Descrição:
Definimos uma função para calcular o WMAPE (Weighted Mean Absolute Percentage Error), uma métrica usada para avaliar a precisão das previsões.

In [16]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

#6. Divisão dos Dados em Treino e Validação

Descrição:

Dividimos os dados em conjuntos de treino e validação. O conjunto de treino inclui dados anteriores a 18 de março de 2024, enquanto o conjunto de validação abrange o período de 18 de março de 2024 a 19 de julho de 2024. Calculamos o horizonte de previsão (h) como o número de dias únicos no conjunto de validação.

In [17]:
train = df_1.loc[df_1['ds']  <  '2024-03-18']
valid = df_1.loc[(df_1['ds'] >= '2024-03-18') & (df_1['ds'] <= '2024-07-19')]
h = valid['ds'].nunique()
h

90

#7. Avaliação dos Modelos de Previsão
Descrição:

Utilizamos os modelos Prophet, AutoARIMA, SeasonalWindowAverage (SeasWA), SeasonalNaive e ETS para realizar previsões em nossos dados de treino e validação. Cada modelo foi ajustado aos dados de treino e utilizamos os dados de validação para calcular as métricas de avaliação (WMAPE, MAE, MSE, R²). Criamos gráficos interativos para visualizar as previsões e os dados reais, incluindo intervalos de confiança quando aplicável.

## Modelo SeasonalNaive



In [18]:
# Ajuste do modelo SeasonalNaive
model_sn = StatsForecast(models=[
                           SeasonalNaive(season_length=22),
                          ],
                         freq='B', n_jobs=-1)
model_sn.fit(train)

# Previsões
forecast_sn = model_sn.predict(h=h)  #
forecast_sn = forecast_sn.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')


# Cálculo e impressão do WMAPE para SeasonalNaive
wmape_sn = wmape(forecast_sn['y'].values, forecast_sn['SeasonalNaive'].values)
MAE_sn = mean_absolute_error(forecast_sn['y'].values, forecast_sn['SeasonalNaive'].values)
MSE_sn = mean_squared_error(forecast_sn['y'].values, forecast_sn['SeasonalNaive'].values)
R2_sn = r2_score(forecast_sn['y'].values, forecast_sn['SeasonalNaive'].values)

print(f'SeasonalNaive WMAPE: {wmape_sn:.2%}')
print(f'SeasonalNaive MAE: {MAE_sn:.3f}')
print(f'SeasonalNaive MSE: {MSE_sn:.4f}')
print(f'SeasonalNaive R2: {R2_sn:.2f}')

# Verifique as colunas disponíveis
print("Colunas disponíveis para SeasonalNaive:", forecast_sn.columns)

# Criar gráfico para SeasonalNaive
fig_sn = go.Figure()
for family in ['ibv']:
    df_family_sn = forecast_sn[forecast_sn['unique_id'] == family]

    # Adiciona a linha real
    fig_sn.add_trace(go.Scatter(x=df_family_sn['ds'], y=df_family_sn['y'],
                                mode='lines',
                                name='Real',
                                line=dict(width=2),
                                hovertext=df_family_sn['ds'].dt.strftime('%Y-%m-%d'),  # Mantendo a data completa no hover
                                hoverinfo='text+y'  # Mostrando a data completa e o valor no hover
                               ))

    # Adiciona a linha do modelo SeasonalNaive
    if 'SeasonalNaive' in df_family_sn.columns:
        fig_sn.add_trace(go.Scatter(x=df_family_sn['ds'], y=df_family_sn['SeasonalNaive'],
                                    mode='lines',
                                    name='SeasonalNaive',
                                    hovertext=df_family_sn['ds'].dt.strftime('%Y-%m-%d'),  # Mantendo a data completa no hover
                                    hoverinfo='text+y'  # Mostrando a data completa e o valor no hover
                                   ))

fig_sn.update_layout(
    title={
        'text': 'SeasonalNaive Forecast',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Mês Ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%m-%Y',  # Exibir mês e ano no formato numérico
        tickangle=-0,  # Melhor legibilidade para as datas
        type='date',
        dtick="M1",  # Define a frequência dos ticks para 1 mês
        nticks=12,  # Garantir que haja ticks suficientes para cada mês
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_sn.show()



SeasonalNaive WMAPE: 2.88%
SeasonalNaive MAE: 3.611
SeasonalNaive MSE: 20.1009
SeasonalNaive R2: -1.50
Colunas disponíveis para SeasonalNaive: Index(['unique_id', 'ds', 'SeasonalNaive', 'y'], dtype='object')


/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:492: FutureWarning:

In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.



##  SeasonalWindowAverage (SeasWA)



In [19]:
# Cálculo e impressão do WMAPE para SeasWA


model_sw = StatsForecast(models=[
                           SeasonalWindowAverage(season_length=22, window_size=2)
                          ],
                         freq='B', n_jobs=-1)
model_sw.fit(train)

# Previsões
forecast_sw = model_sw.predict(h=h)
forecast_sw = forecast_sw.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

# Calcular métricas de erro
wmape_wa = wmape(forecast_sw['y'].values, forecast_sw['SeasWA'].values)
MAE_wa = mean_absolute_error(forecast_sw['y'].values, forecast_sw['SeasWA'].values)
MSE_wa = mean_squared_error(forecast_sw['y'].values, forecast_sw['SeasWA'].values)
R2_wa = r2_score(forecast_sw['y'].values, forecast_sw['SeasWA'].values)

print(f'SeasWA WMAPE: {wmape_wa:.2%}')
print(f'SeasWA MAE: {MAE_wa:.3f}')
print(f'SeasWA MSE: {MSE_wa:.4f}')
print(f'SeasWA R2: {R2_wa:.2f}')

# Verifique as colunas disponíveis
print("Colunas disponíveis para SeasWA:", forecast_sw.columns)

# Criar gráfico para SeasWA
fig_sw = go.Figure()
for family in ['ibv']:
    df_family_sw = forecast_sw[forecast_sw['unique_id'] == family]

    # Adiciona a linha real
    fig_sw.add_trace(go.Scatter(x=df_family_sw['ds'], y=df_family_sw['y'],
                                mode='lines',
                                name='Real',
                                line=dict(width=2),
                                hovertext=df_family_sw['ds'].dt.strftime('%Y-%m-%d'),  # Mantendo a data completa no hover
                                hoverinfo='text+y'  # Mostrando a data completa e o valor no hover
                               ))

    # Adiciona a linha do modelo SeasWA
    if 'SeasWA' in df_family_sw.columns:
        fig_sw.add_trace(go.Scatter(x=df_family_sw['ds'], y=df_family_sw['SeasWA'],
                                    mode='lines',
                                    name='SeasWA',
                                    hovertext=df_family_sw['ds'].dt.strftime('%Y-%m-%d'),  # Mantendo a data completa no hover
                                    hoverinfo='text+y'  # Mostrando a data completa e o valor no hover
                                   ))

fig_sw.update_layout(
    title={
        'text': 'SeasWA Forecast',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Mês Ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%m-%Y',  # Exibir mês e ano no formato numérico
        tickangle=-0,  # Melhor legibilidade para as datas
        type='date',
        dtick="M1",  # Define a frequência dos ticks para 1 mês
        nticks=12,  # Garantir que haja ticks suficientes para cada mês
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_sw.show()

SeasWA WMAPE: 2.54%
SeasWA MAE: 3.186
SeasWA MSE: 16.8238
SeasWA R2: -1.09
Colunas disponíveis para SeasWA: Index(['unique_id', 'ds', 'SeasWA', 'y'], dtype='object')


/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:492: FutureWarning:

In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.



## ETS (Exponential Smoothing)


In [20]:
season_length = 252  # Exemplo para dados diários com sazonalidade anual


# Ajustar o modelo ETS (Exponential Smoothing)
model_ets = sm.tsa.ExponentialSmoothing(train['y'], seasonal='mul', seasonal_periods=season_length).fit()

# Fazer previsões
forecast_ets = model_ets.forecast(h)
forecast_dates = pd.date_range(start=train['ds'].iloc[-1] + pd.Timedelta(days=1), periods=h, freq='B')
ets_df = pd.DataFrame({'ds': forecast_dates, 'ets': forecast_ets})
ets_df = ets_df.merge(valid, on=['ds'], how='inner')

# Calcular métricas de erro
wmape_ets = wmape(ets_df['y'].values, ets_df['ets'].values)
MAE_ets = mean_absolute_error(ets_df['y'].values, ets_df['ets'].values)
MSE_ets = mean_squared_error(ets_df['y'].values, ets_df['ets'].values)
R2_ets = r2_score(ets_df['y'].values, ets_df['ets'].values)
print(f'ETS WMAPE: {wmape_ets:.2%}')
print(f'ETS MAE: {MAE_ets:.3f}')
print(f'ETS MSE: {MSE_ets:.4f}')
print(f'ETS R2: {R2_ets:.2f}')

# Visualizar as previsões com plotly
fig_ets = go.Figure()

# Adicionar a linha observada
fig_ets.add_trace(go.Scatter(x=train['ds'], y=train['y'],
                             mode='lines', name='Observedo',
                             line=dict(width=2),
                             hovertext=train['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar a linha realizada
fig_ets.add_trace(go.Scatter(x=valid['ds'], y=valid['y'],
                             mode='lines', name='Realizado',
                             line=dict(width=2),
                             hovertext=valid['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar a linha da previsão ETS
fig_ets.add_trace(go.Scatter(x=ets_df['ds'], y=ets_df['ets'],
                             mode='lines', name='Forecast',
                             line=dict(width=2),
                             hovertext=ets_df['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

fig_ets.update_layout(
    title={
        'text': 'ETS Forecast',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%Y',  # Exibir apenas o ano no eixo x
        tickangle=-0,  # Melhor legibilidade para as datas
        type='date',
        dtick="M12",  # Define a frequência dos ticks para 1 ano

    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_ets.show()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



ETS WMAPE: 2.91%
ETS MAE: 3.659
ETS MSE: 20.3983
ETS R2: -1.54


In [21]:
ets_df

,ds,ets,unique_id,y
0,2024-03-18,127.345631,ibv,126.954
1,2024-03-19,126.646192,ibv,127.529
2,2024-03-20,126.196008,ibv,129.125
3,2024-03-21,126.504705,ibv,128.159
4,2024-03-22,125.454533,ibv,127.027
...,...,...,...,...
85,2024-07-15,130.326903,ibv,129.321
86,2024-07-16,130.570477,ibv,129.110
87,2024-07-17,131.595488,ibv,129.450
88,2024-07-18,132.152041,ibv,127.652


## Modelo Prophet


In [22]:
m = Prophet(yearly_seasonality=True)
m.fit(train)

future = m.make_future_dataframe(periods=len(valid), freq='B')
prophet_df = m.predict(future)


prophet_df = prophet_df[['ds', 'yhat']].merge(valid, on='ds', how='inner')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj89o5t_o/ro9rhlyz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj89o5t_o/dwyspc6z.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19286', 'data', 'file=/tmp/tmpj89o5t_o/ro9rhlyz.json', 'init=/tmp/tmpj89o5t_o/dwyspc6z.json', 'output', 'file=/tmp/tmpj89o5t_o/prophet_modelav6feibh/prophet_model-20240728141728.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:17:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:17:30 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [23]:
m = Prophet(yearly_seasonality=True)
m.fit(train)

future = m.make_future_dataframe(periods=len(valid), freq='B')
prophet_df = m.predict(future)


prophet_df = prophet_df[['ds', 'yhat']].merge(valid, on='ds', how='inner')

# Calcular métricas de erro
wmape_pr = wmape(prophet_df['y'].values, prophet_df['yhat'].values)
MAE_pr = mean_absolute_error(prophet_df['y'].values, prophet_df['yhat'].values)
MSE_pr = mean_squared_error(prophet_df['y'].values, prophet_df['yhat'].values)
R2_pr = r2_score(prophet_df['y'].values, prophet_df['yhat'].values)
print(f'Prophet WMAPE: {wmape_pr:.2%}')
print(f'Prophet MAE: {MAE_pr:.3f}')
print(f'Prophet MSE: {MSE_pr:.4f}')
print(f'Prophet R2: {R2_pr:.2f}')

# Visualizar as previsões com plotly
fig_pr = go.Figure()

# Adicionar a linha observada
fig_pr.add_trace(go.Scatter(x=train['ds'], y=train['y'],
                            mode='lines', name='Observado',
                            line=dict(width=2),
                            hovertext=train['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'))

# Adicionar a linha realizada
fig_pr.add_trace(go.Scatter(x=valid['ds'], y=valid['y'],
                            mode='lines', name='Realizado',
                            line=dict(width=2),
                            hovertext=valid['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'))

# Adicionar a linha da previsão Prophet
fig_pr.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['yhat'],
                            mode='lines', name='Previsão',
                            line=dict(width=2),
                            hovertext=prophet_df['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'))

fig_pr.update_layout(
    title={
        'text': 'Previsão Prophet',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%Y',  # Exibir apenas o ano no eixo x
        tickangle=-0,  # Melhor legibilidade para as datas
        type='date',
        dtick="M12",  # Define a frequência dos ticks para 1 ano
        tickmode='linear',  # Usa o modo de tick linear
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_pr.show()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj89o5t_o/8df0uutr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj89o5t_o/7_07b1a_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50322', 'data', 'file=/tmp/tmpj89o5t_o/8df0uutr.json', 'init=/tmp/tmpj89o5t_o/7_07b1a_.json', 'output', 'file=/tmp/tmpj89o5t_o/prophet_model3kccqs0k/prophet_model-20240728141731.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:17:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:17:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet WMAPE: 5.21%
Prophet MAE: 6.547
Prophet MSE: 63.5054
Prophet R2: -6.90


In [24]:
prophet_df

,ds,yhat,unique_id,y
0,2024-03-18,121.346451,ibv,126.954
1,2024-03-19,121.346587,ibv,127.529
2,2024-03-20,121.279701,ibv,129.125
3,2024-03-21,121.218940,ibv,128.159
4,2024-03-22,121.165784,ibv,127.027
...,...,...,...,...
85,2024-07-15,135.918936,ibv,129.321
86,2024-07-16,136.326304,ibv,129.110
87,2024-07-17,136.615180,ibv,129.450
88,2024-07-18,136.854722,ibv,127.652


## AutoARIMA


In [25]:
# Treinamento do modelo AutoARIMA
model = StatsForecast(models=[AutoARIMA(season_length=22)], freq='B', n_jobs=-1)
model.fit(train)

# Previsão
h = valid['ds'].nunique()
arima_df = model.predict(h=h, level=[90])
arima_df = arima_df.merge(valid, on=['ds', 'unique_id'], how='inner')

# Calcular métricas de erro
wmape_ari = wmape(arima_df['y'].values, arima_df['AutoARIMA'].values)
MAEari = mean_absolute_error(arima_df['y'].values, arima_df['AutoARIMA'].values)
MSEari = mean_squared_error(arima_df['y'].values, arima_df['AutoARIMA'].values)
R2ari = r2_score(arima_df['y'].values, arima_df['AutoARIMA'].values)
print(f'AutoArima WMAPE: {wmape_ari:.2%}')
print(f'AutoArima MAE: {MAEari}')
print(f'AutoArima MSE: {MSEari:.4f}')
print(f'AutoArima R2: {R2ari:.2f}')

# Visualizar as previsões com plotly
fig_ari = go.Figure()

# Adicionar a linha observada
fig_ari.add_trace(go.Scatter(x=train['ds'], y=train['y'],
                             mode='lines', name='Observado',
                             line=dict(width=2, color='blue'),
                             hovertext=train['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar a linha realizada
fig_ari.add_trace(go.Scatter(x=valid['ds'], y=valid['y'],
                             mode='lines', name='Realizado',
                             line=dict(width=2, color='green'),
                             hovertext=valid['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar a linha da previsão AutoARIMA
fig_ari.add_trace(go.Scatter(x=arima_df['ds'], y=arima_df['AutoARIMA'],
                             mode='lines', name='Previsão',
                             line=dict(width=2, color='red'),
                             hovertext=arima_df['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar os intervalos de confiança
fig_ari.add_trace(go.Scatter(
    x=arima_df['ds'], y=arima_df['AutoARIMA-lo-90'],
    mode='lines', name='Intervalo Inferior (lo-90)',
    line=dict(width=1, dash='dash', color='purple'),
    hovertext=arima_df['ds'].dt.strftime('%Y-%m-%d'),
    hoverinfo='text+y'
))

fig_ari.add_trace(go.Scatter(
    x=arima_df['ds'], y=arima_df['AutoARIMA-hi-90'],
    mode='lines', name='Intervalo Superior (hi-90)',
    line=dict(width=1, dash='dash', color='orange'),
    hovertext=arima_df['ds'].dt.strftime('%Y-%m-%d'),
    hoverinfo='text+y',
    fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'  # Mudança na cor de preenchimento para laranja claro
))

fig_ari.update_layout(
    title={
        'text': 'Previsão AutoARIMA com Intervalos de Confiança',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Data',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%Y',  # Exibir apenas o ano no eixo x
        tickangle=-0,  # Melhor legibilidade para as datas
        type='date',
        dtick="M12",  # Define a frequência dos ticks para 1 ano
        tickmode='linear',  # Usa o modo de tick linear
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_ari.show()

AutoArima WMAPE: 1.87%
AutoArima MAE: 2.3457765204535583
AutoArima MSE: 9.3745
AutoArima R2: -0.17


/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:492: FutureWarning:

In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.



In [26]:
arima_df

,ds,unique_id,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,y
0,2024-03-18,ibv,126.850739,124.238998,129.462479,126.954
1,2024-03-19,ibv,126.773567,123.288719,130.258408,127.529
2,2024-03-20,ibv,126.746269,122.468071,131.024475,129.125
3,2024-03-21,ibv,126.750175,121.763985,131.736359,128.159
4,2024-03-22,ibv,126.744843,121.145531,132.344147,127.027
...,...,...,...,...,...,...
85,2024-07-15,ibv,126.744125,103.007240,150.481018,129.321
86,2024-07-16,ibv,126.744125,102.869263,150.618988,129.110
87,2024-07-17,ibv,126.744125,102.732086,150.756165,129.450
88,2024-07-18,ibv,126.744125,102.595680,150.892563,127.652


# Conclusão:


Neste projeto, utilizamos cinco modelos diferentes para realizar previsões em séries temporais de dados financeiros: Prophet, AutoARIMA, SeasonalWindowAverage (SeasWA), SeasonalNaive e ETS (Exponential Smoothing). Cada modelo foi ajustado aos dados de treino, e utilizamos os dados de validação para calcular as métricas de avaliação, incluindo WMAPE, MAE, MSE e R².

Para cada modelo, criamos gráficos interativos usando Plotly para visualizar as previsões e compará-las com os dados reais. Essas visualizações ajudaram a identificar o desempenho de cada modelo e a eficácia das previsões realizadas.

Os resultados das métricas de avaliação foram os seguintes:



*   **SeasonalNaive:**

   *   WMAPE: 2.88%
   *   MAE: 3.611
   *   MSE: 20.1009
   *   R²: -1.50
   *   Apesar de sua simplicidade, o modelo SeasonalNaive demonstrou previsões
       relativamente precisas, com um WMAPE razoável.




* **SeasonalWindowAverage (SeasWA):**

    * WMAPE: 2.54%
    * MAE: 3.186
    * MSE: 16.8238
    * R²: -1.09
    * SeasWA superou o SeasonalNaive, mostrando que uma média deslizante sazonal
      pode captar padrões de forma mais eficaz.



* **ETS (Exponential Smoothing):**

    * WMAPE: 2.91%
    * MAE: 3.659
    * MSE: 20.3983
    * R²: -1.54
    * O modelo ETS apresentou um desempenho comparável ao SeasonalNaive, mas não se destacou em relação aos outros modelos.


* **Prophet:**

  * WMAPE: 5.21%
  * MAE: 6.547
  * MSE: 63.5054
  * R²: -6.90
  * Apesar de sua popularidade, o Prophet não conseguiu captar com precisão as
    tendências dos dados financeiros neste caso específico.


* **AutoARIMA:**

  * WMAPE: 1.87%
  * MAE: 2.3457765204535583
  * MSE: 9.3745
  * R²: -0.17
  * AutoARIMA destacou-se como o modelo mais preciso, com o menor WMAPE e MAE, mostrando sua robustez na previsão de séries temporais financeiras.



As análises e previsões realizadas com esses modelos fornecem uma compreensão abrangente das tendências e padrões nos dados financeiros, auxiliando na tomada de decisões informadas e estratégicas. A eficácia dos modelos varia, mas AutoARIMA emergiu como a escolha superior para este conjunto de dados, destacando-se em termos de precisão preditiva.